In [1]:
import os
import pandas as pd

from dbfread import DBF

sys.path.insert(0, '..\..\Resources\Python-Functions')
import BigQuery

# import transit route outputs
bqclient = BigQuery.getBigQueryClient_TDMScenarios()

In [82]:
dfScenarios = bqclient.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.scenarios").to_dataframe()
dfScenarios

,scenarioID,tdmVersion,tdmVersionWithDate,scenarioName,scenarioGroup,scenarioYear
0,1,v832,WF TDM v8.3.2 - 2022-02-04a,BY_2019,BY,2019
1,2,v832,WF TDM v8.3.2 - 2022-02-04a,RTP_2030,RTP,2030
2,3,v832,WF TDM v8.3.2 - 2022-02-04a,RTP_2040,RTP,2040
3,4,v832,WF TDM v8.3.2 - 2022-02-04a,RTP_2050,RTP,2050
4,5,v832,WF TDM v8.3.2 - 2022-02-04a,TIP_2024,TIP,2024
5,6,v832,WF TDM v8.3.2 - 2022-09-21,BY_2019,BY,2019
6,7,v832,WF TDM v8.3.2 - 2022-09-21,BY_2023,BY,2023
7,8,v832,WF TDM v8.3.2 - 2022-09-21,Needs_2032,Needs,2032
8,9,v832,WF TDM v8.3.2 - 2022-09-21,Needs_2042,Needs,2042
9,10,v832,WF TDM v8.3.2 - 2022-09-21,Needs_2050,Needs,2050


In [3]:
dfODDetail = bqclient.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.route_od_detail").to_dataframe()
#display(dfODDetail)
dfPADetail = bqclient.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.route_pa_detail").to_dataframe()
#display(dfPADetail)

In [4]:
# read in manully created file
dfRouteSummaryNames_Revised = pd.read_csv(r"E:\GitHub\TDM-Scenario-Comparison-Toolbox\0-ModelModifications\manual\routesummarynames_revised.csv")
#dfRouteSummaryNames_Revised

In [5]:
paDropColumns = ['NAME',
                 'LONGNAME'     ,
                 'numRecords'   ,
                 'minYear'      ,
                 'maxYear'      ,
                 'minMode'      ,
                 'NAMEwithZeros',
                 'ROUTEID'      ,
                 'NAMEID'       ,
                 'OPERATOR'     ,
                 'PK_HEADWAY'   ,
                 'OK_HEADWAY'   ,
                 'PK_TIME'      ,
                 'OK_TIME'      ,
                 'DIST'         ,
                 'PK_D_BRDA'    ,
                 'PK_W_BRDA'    ,
                 'PK_BRDA'      ,
                 'PK_XITB'      ,
                 'OK_D_BRDA'    ,
                 'OK_W_BRDA'    ,
                 'OK_BRDA'      ,
                 'OK_XITB'      ,
                 'DY_D4_BRDA'   ,
                 'DY_W4_BRDA'   ,
                 'DY_4_XITB'    ,
                 'DY_D5_BRDA'   ,
                 'DY_W5_BRDA'   ,
                 'DY_5_XITB'    ,
                 'DY_D6_BRDA'   ,
                 'DY_W6_BRDA'   ,
                 'DY_6_XITB'    ,
                 'DY_D7_BRDA'   ,
                 'DY_W7_BRDA'   ,
                 'DY_7_XITB'    ,
                 'DY_D8_BRDA'   ,
                 'DY_W8_BRDA'   ,
                 'DY_8_XITB'    ,
                 'DY_D9_BRDA'   ,
                 'DY_W9_BRDA'   ,
                 'DY_9_XITB'    ,
                 'DY_D_BRDA'    ,
                 'DY_W_BRDA'    ,
                 'DY_BRDA'      ,
                 'DY_XITB'      ]


In [6]:
dfODDetailWithRevNames = pd.DataFrame.merge(dfODDetail,dfRouteSummaryNames_Revised,on='NAME')
dfODDetailWithRevNames = dfODDetailWithRevNames.drop(columns=('NAME'))
dfODDetailWithRevNames = dfODDetailWithRevNames.rename(columns={'NAME_New':'NAME'})
#display(dfODDetailWithRevNames)

# change negative to column
dfPADetail.loc[(dfPADetail['NAME'].str.strip().str[-1]!='-'), 'Direction'] = 'P'
dfPADetail.loc[(dfPADetail['NAME'].str.strip().str[-1]=='-'), 'Direction'] = 'N'
dfPADetail.loc[(dfPADetail['NAME'].str.strip().str[-1]=='-'), 'NAME'] = dfPADetail['NAME'].str[:-1]

dfPADetailWithRevNames = pd.DataFrame.merge(dfPADetail,dfRouteSummaryNames_Revised,on='NAME')
dfPADetailWithRevNames = dfPADetailWithRevNames.drop(columns=(paDropColumns))
dfPADetailWithRevNames = dfPADetailWithRevNames.rename(columns={'NAME_New':'NAME'})
#display(dfPADetailWithRevNames)

In [7]:
dfTotVol = pd.DataFrame()

# loop through scenarios and calc Link A and B -- USE Transit-Map TransitRidership.ipynb methodology
for index, row in dfScenarios.iterrows():

    dfTemp = dfODDetailWithRevNames[dfODDetailWithRevNames['scenarioID']==row['scenarioID']]
    #display(dfTemp)

    dfLinkLookup = dfTemp[['scenarioID','MODE','NAMEID','NAME','LINKSEQ1','A']].copy()
    dfLinkLookup['LINKSEQ_OF_B'] = dfLinkLookup['LINKSEQ1'] + 1

    dfLinkLookupMax = dfLinkLookup.groupby(['scenarioID','NAMEID','NAME'],as_index=False).agg(MAXLINKSEQ1=('LINKSEQ1','max'))
    dfLinkLookupMax['MAXLINKSEQp1'] = dfLinkLookupMax['MAXLINKSEQ1'] + 1
    dfLinkLookupWithMaxPlus1 = pd.DataFrame.merge(dfLinkLookup, dfLinkLookupMax, on=('scenarioID','NAMEID','NAME'), how='left')
    dfLinkLookupWithMaxPlus1.loc[(dfLinkLookupWithMaxPlus1['LINKSEQ1']==0), 'LINKSEQ1'] = dfLinkLookupWithMaxPlus1['MAXLINKSEQp1']
    dfLinkLookupWithMaxPlus1.loc[(dfLinkLookupWithMaxPlus1['LINKSEQ_OF_B']==1), 'LINKSEQ_OF_B'] = 0

    dfLinkLookupWithMaxPlus1 = dfLinkLookupWithMaxPlus1.drop(columns=['MAXLINKSEQ1','MAXLINKSEQp1'])

    dfJoinToGetB = pd.DataFrame.merge(dfLinkLookupWithMaxPlus1, dfLinkLookupWithMaxPlus1, left_on=('MODE','NAMEID','NAME','LINKSEQ_OF_B'), right_on=('MODE','NAMEID','NAME','LINKSEQ1'))
    dfJoinToGetB = dfJoinToGetB[['MODE','NAMEID','NAME','LINKSEQ1_x','A_x','A_y']]
    dfJoinToGetB.columns = ('MODE','NAMEID','NAME','LINKSEQ1','A','B')

    dfTempPrepped = pd.DataFrame.merge(dfJoinToGetB,dfTemp[['MODE','NAMEID','NAME','LINKSEQ1','A','DIST','D_TOT_VOL','scenarioID']], on=('MODE','NAMEID','NAME','LINKSEQ1','A'))

    #display(dfLinkLookup)
    #display(dfLinkLookupMax)
    #display(dfLinkLookupWithMaxPlus1)
    #display(dfJoinToGetB)
    #display(dfTempPrepped)

    dfTotVol = dfTotVol.append(dfTempPrepped)

    #display(dfLinkLookup.sort_values(by='LINK', ascending=False).drop_duplicates(subset='NAME'))

display(dfTotVol)
display(dfTotVol['D_TOT_VOL'].sum())

,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,scenarioID
0,4,4.0,DA605F,1,20131,3551,0.11,114.99,1
1,4,4.0,DA605F,6,8979,3411,0.09,119.31,1
2,4,4.0,DA605F,10,3415,9693,0.18,91.33,1
3,4,4.0,DA605F,11,9693,3417,0.17,78.85,1
4,4,4.0,DA605F,12,3417,3570,0.14,70.61,1
...,...,...,...,...,...,...,...,...,...
8223,4,16.0,UT809F,94,68565,68576,0.12,13.81,32
8224,4,16.0,UT809F,95,68576,68608,0.10,13.81,32
8225,4,16.0,UT809F,96,68608,68651,0.10,13.81,32
8226,4,16.0,UT809F,97,68651,68670,0.09,13.81,32


176933078.59000003

In [68]:
# Add SegIDs
#dfNetWithSegIDs = pd.DataFrame(DBF(r"A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-12-19\1_Inputs\3_Highway\GIS\MasterNet_v9 - 2023-01-26 - Link.dbf", load=True))
dfNetWithSegIDs = pd.DataFrame(DBF(r"A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-12-19\Scenarios\Needs_2050\0_InputProcessing\v9_Needs_SE50_Net50_Link.DBF", load=True))
dfNetWithSegIDs = dfNetWithSegIDs[['A','B','SEGID','IB_OB']]
dfNetWithSegIDs = dfNetWithSegIDs[dfNetWithSegIDs['SEGID']!='']
dfNetWithSegIDs

,A,B,SEGID,IB_OB
8479,15115,25206,WFRC_8225,OB
8481,15115,25263,WFRC_8224,IB
8486,15120,25281,WFRC_8227,OB
8487,15120,25332,WFRC_8226,IB
8511,20001,20002,2688_009.7,IB
...,...,...,...,...
50051,71603,71596,2899_004.1,IB
50054,71604,67241,MAG_6145,IB
50055,71605,67274,MAG_6168,OB
50058,71607,62992,MAG_6157,IB


In [69]:
# join od detail with link seg data. only preserve rows with data in both tables since some links don't have segids etc
dfTotVolWithSegIDs = pd.DataFrame.merge(dfTotVol,dfNetWithSegIDs,on=('A','B'),how='inner')
dfTotVolWithSegIDs['DISTxVOL'] = dfTotVolWithSegIDs['DIST'] * dfTotVolWithSegIDs['D_TOT_VOL']
dfTotVolWithSegIDs

,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,scenarioID,SEGID,IB_OB,DISTxVOL
0,4,13.0,UT834,37,23870,23869,0.24,128.85,1,2218_000.5,IB,30.9240
1,4,30.0,MAG_017,58,23870,23869,0.24,101.89,2,2218_000.5,IB,24.4536
2,4,31.0,MAG_018,35,23870,23869,0.24,130.05,2,2218_000.5,IB,31.2120
3,4,30.0,MAG_017,56,23870,23869,0.24,114.70,3,2218_000.5,IB,27.5280
4,4,31.0,MAG_018,35,23870,23869,0.24,144.72,3,2218_000.5,IB,34.7328
...,...,...,...,...,...,...,...,...,...,...,...,...
187535,4,7.0,WFRC_037,135,62338,62341,0.51,35.17,25,0145_003.8,IB,17.9367
187536,4,1.0,WFRC_037,136,62341,71513,0.46,33.60,24,0145_004.4,IB,15.4560
187537,4,7.0,WFRC_037,136,62341,71513,0.46,35.17,25,0145_004.4,IB,16.1782
187538,4,1.0,WFRC_037,137,71513,62346,0.15,33.60,24,0145_004.8,IB,5.0400


In [70]:
dfTotVolStatsBySegID = dfTotVolWithSegIDs.groupby(['scenarioID','MODE','NAME','SEGID'],as_index=False).agg(DISTxVOL=('DISTxVOL','sum'),DIST=('DIST','sum'))
dfTotVolStatsBySegID['SEGVOL'] = round(dfTotVolStatsBySegID['DISTxVOL'] / dfTotVolStatsBySegID['DIST'],0).astype(int)
dfTotVolStatsBySegID.drop(columns=['DISTxVOL','DIST'],inplace=True)
dfTotVolStatsBySegID

,scenarioID,MODE,NAME,SEGID,SEGVOL
0,1,4,UT806,WFRC_8222,58
1,1,4,UT834,2218_000.5,129
2,1,4,UT862,2080_005.7,201
3,1,7,701_Blue,2030_002.7,5434
4,1,7,701_Blue,2030_004.6,7362
...,...,...,...,...,...
62074,32,9,WE603,3400_000.1,1151
62075,32,9,WE603,3400_000.3,1171
62076,32,9,WE603,3400_000.4,1423
62077,32,9,WE603,3400_000.6,1423


In [71]:
dfTotVolStatsBySegID_byMode = dfTotVolStatsBySegID.groupby(['scenarioID','SEGID','MODE'], as_index=False).agg(SEGVOLBYMODE=('SEGVOL','sum'))
dfTotVolStatsBySegID_byMode = dfTotVolStatsBySegID_byMode.pivot_table(index=('scenarioID','SEGID'), columns='MODE')
dfTotVolStatsBySegID_byMode.fillna(0,inplace=True)
dfTotVolStatsBySegID_byMode = dfTotVolStatsBySegID_byMode.astype(int)
dfTotVolStatsBySegID_byMode = dfTotVolStatsBySegID_byMode.reset_index()
dfTotVolStatsBySegID_byMode = dfTotVolStatsBySegID_byMode.droplevel(0, axis=1) 
dfTotVolStatsBySegID_byMode.columns = ['scenarioID','SEGID','M4','M5','M6','M7','M8','M9']
dfTotVolStatsBySegID_byMode['MT'] =  dfTotVolStatsBySegID_byMode['M4'] + \
                                     dfTotVolStatsBySegID_byMode['M5'] + \
                                     dfTotVolStatsBySegID_byMode['M6'] + \
                                     dfTotVolStatsBySegID_byMode['M7'] + \
                                     dfTotVolStatsBySegID_byMode['M8'] + \
                                     dfTotVolStatsBySegID_byMode['M9']
dfTotVolStatsBySegID_byMode

,scenarioID,SEGID,M4,M5,M6,M7,M8,M9,MT
0,1,0048_004.1,0,0,0,0,0,3124,3124
1,1,2030_002.7,0,0,0,12550,0,0,12550
2,1,2030_004.6,0,0,0,14565,0,0,14565
3,1,2030_005.1,0,0,0,4166,0,0,4166
4,1,2031_002.2,0,0,0,3134,0,0,3134
...,...,...,...,...,...,...,...,...,...
39032,32,WFRC_8227,562,0,0,0,0,0,562
39033,32,WFRC_8256,34,0,0,0,0,0,34
39034,32,WFRC_8258,100,0,0,0,0,0,100
39035,32,WFRC_8260,115,0,0,0,0,0,115


In [72]:
# read in json ids
dfWABScenarios = pd.read_json(r"E:\WebAppBuilderForArcGIS_v2.20\server\apps\8\widgets\tttScenarioManager\data\scenarios.json")
#display(dfWABScenarios)

# match with BigQuery Scenarios

dfBigQueryToWAB = pd.DataFrame([
    [1 , 'v8.3.2 BY 2019'       ],
    [2 , 'v8.3.2 RTP 2030'      ],
    [3 , 'v8.3.2 RTP 2040'      ],
    [4 , 'v8.3.2 RTP 2050'      ],
    [5 , 'v8.3.2 TIP 2024'      ],
    [18, 'v9 Beta BY 2019'      ],
    [19, 'v9 Beta BY 2023'      ],
    [20, 'v9 Beta RTP+EF 2032'  ],
    [21, 'v9 Beta RTP+EF 2042'  ],
    [22, 'v9 Beta RTP+EF 2050'  ],
    [23, 'v9 Beta Needs 2032'   ],
    [24, 'v9 Beta Needs 2042'   ],
    [25, 'v9 Beta Needs 2050'   ],
    [26, 'v9 Beta No Build 2032'],
    [27, 'v9 Beta No Build 2042'],
    [28, 'v9 Beta No Build 2050'],
    [29, 'v9 Beta RTP 2032'     ],
    [30, 'v9 Beta RTP 2042'     ],
    [31, 'v9 Beta RTP 2050'     ],
    [32, 'v9 Beta TIP 2028'     ]
],columns=('scenarioID', 'label'))
#display(dfBigQueryToWAB)

dfJoin = pd.DataFrame.merge(dfBigQueryToWAB,dfWABScenarios,on='label',how='left')
dfJoin = dfJoin[['scenarioID','value']]
dfJoin

,scenarioID,value
0,1,id_12066090786226718a9b4755.54851854
1,2,id_602780967622672681e6327.84737895
2,3,id_4927651456226727eb56079.08455813
3,4,id_16231621306226729673c147.07236076
4,5,id_3989814362267228a9f4b5.00876734
5,18,id_37001244863cadd40e1b8f7.52642706
6,19,id_72534583163cb54226ea202.98630416
7,20,id_73730688563cb5d2222cdc4.25550859
8,21,id_71963308763cb5d35348012.40299726
9,22,id_23888985563cb5d49295ba5.99558740


In [73]:
dfJoin[dfJoin['scenarioID']==2].value

1    id_602780967622672681e6327.84737895
Name: value, dtype: object

In [74]:
export = False

_outputdir_mode = os.path.join(os.getcwd(),'TTTDataTransitDetailByMode')
_outputdir_route = os.path.join(os.getcwd(),'TTTDataTransitDetailByRoute')

# loop through scenarios and calc Link A and B -- USE Transit-Map TransitRidership.ipynb methodology
for index, row in dfScenarios.iterrows():

    try:
        #print( dfJoin[dfJoin['scenarioID']==row['scenarioID']]['value'].tolist())
        _id = dfJoin[dfJoin['scenarioID']==row['scenarioID']]['value'].tolist()[0]
        #print(_id)
        _file = _id + '.json'
        #print(_file)
        _outputfile_mode = os.path.join(_outputdir_mode,_file)
        _outputfile_route = os.path.join(_outputdir_route,_file)

        dfModeOut = dfTotVolStatsBySegID_byMode[dfTotVolStatsBySegID_byMode['scenarioID']==row['scenarioID']]
        dfModeOut = dfModeOut.drop(columns=('scenarioID'))
        dfModeOut = dfModeOut.set_index('SEGID')
        if export: dfModeOut.to_json(_outputfile_mode,orient="table") #,orient='records'

        dfRouteOut = dfTotVolStatsBySegID[dfTotVolStatsBySegID['scenarioID']==row['scenarioID']]
        dfRouteOut = dfRouteOut.drop(columns=('scenarioID'))
        dfRouteOut = dfRouteOut.set_index('SEGID')
        if export: dfRouteOut.to_json(_outputfile_route,orient="table") #,orient='records'
        #display(dfTemp)
    except:
        print('scenarioID ' + str(row['scenarioID']) + ' error')
        continue

scenarioID 6 error
scenarioID 7 error
scenarioID 8 error
scenarioID 9 error
scenarioID 10 error
scenarioID 11 error
scenarioID 12 error
scenarioID 13 error
scenarioID 14 error
scenarioID 15 error
scenarioID 16 error
scenarioID 17 error


In [75]:
dfTotVolStatsBySegID_byMode[dfTotVolStatsBySegID_byMode['SEGID']=='WFRC_8227']

,scenarioID,SEGID,M4,M5,M6,M7,M8,M9,MT
1543,6,WFRC_8227,834,0,0,0,0,0,834
2832,7,WFRC_8227,926,0,0,0,0,0,926
4194,8,WFRC_8227,633,0,0,0,0,0,633
5602,9,WFRC_8227,611,0,0,0,0,0,611
7088,10,WFRC_8227,678,0,0,0,0,0,678
8549,11,WFRC_8227,1098,0,0,0,0,0,1098
10009,12,WFRC_8227,1318,0,0,0,0,0,1318
11469,13,WFRC_8227,1412,0,0,0,0,0,1412
12967,14,WFRC_8227,640,0,0,0,0,0,640
14517,15,WFRC_8227,601,0,0,0,0,0,601


In [76]:
dfPADetail_melt = dfPADetailWithRevNames
dfPADetail_melt = dfPADetail_melt.melt(id_vars=['scenarioID','NAME','Direction','MODE','LINKSEQ','A','B','STOPA','STOPB'], var_name='TransitVariable', value_name='TransitValue')
dfPADetail_melt
#dfPADetail_melt.columns

,scenarioID,NAME,Direction,MODE,LINKSEQ,A,B,STOPA,STOPB,TransitVariable,TransitValue
0,1,DA461,P,4,16,3423,3579,1,1,PK_D4_BRDA,0.0
1,1,DA461,P,4,17,3579,5516,1,1,PK_D4_BRDA,0.0
2,1,DA461,P,4,49,6784,3580,1,1,PK_D4_BRDA,0.0
3,1,DA461,P,4,64,3463,3486,1,1,PK_D4_BRDA,0.0
4,1,DA461,P,4,159,3486,3463,1,1,PK_D4_BRDA,0.0
...,...,...,...,...,...,...,...,...,...,...,...
14059867,31,MAG_002,N,9,14,63466,67894,0,0,OK_9_XITB,0.0
14059868,31,MAG_002,N,9,15,67894,67873,0,0,OK_9_XITB,0.0
14059869,31,MAG_002,N,9,16,67873,67863,0,0,OK_9_XITB,0.0
14059870,31,MAG_002,N,9,17,67863,67858,0,0,OK_9_XITB,0.0


In [77]:
dfVarToFields = pd.DataFrame([
 ['PK_D4_BRDA'   , 'PK', 'D', 4, 'BRDA'],
 ['PK_W4_BRDA'   , 'PK', 'W', 4, 'BRDA'],
 ['PK_4_XITB'    , 'PK', 'X', 4, 'XITB'],
 ['PK_D5_BRDA'   , 'PK', 'D', 5, 'BRDA'],
 ['PK_W5_BRDA'   , 'PK', 'W', 5, 'BRDA'],
 ['PK_5_XITB'    , 'PK', 'X', 5, 'XITB'],
 ['PK_D6_BRDA'   , 'PK', 'D', 6, 'BRDA'],
 ['PK_W6_BRDA'   , 'PK', 'W', 6, 'BRDA'],
 ['PK_6_XITB'    , 'PK', 'X', 6, 'XITB'],
 ['PK_D7_BRDA'   , 'PK', 'D', 7, 'BRDA'],
 ['PK_W7_BRDA'   , 'PK', 'W', 7, 'BRDA'],
 ['PK_7_XITB'    , 'PK', 'X', 7, 'XITB'],
 ['PK_D8_BRDA'   , 'PK', 'D', 8, 'BRDA'],
 ['PK_W8_BRDA'   , 'PK', 'W', 8, 'BRDA'],
 ['PK_8_XITB'    , 'PK', 'X', 8, 'XITB'],
 ['PK_D9_BRDA'   , 'PK', 'D', 9, 'BRDA'],
 ['PK_W9_BRDA'   , 'PK', 'W', 9, 'BRDA'],
 ['PK_9_XITB'    , 'PK', 'X', 9, 'XITB'],
 ['OK_D4_BRDA'   , 'OK', 'D', 4, 'BRDA'],
 ['OK_W4_BRDA'   , 'OK', 'W', 4, 'BRDA'],
 ['OK_4_XITB'    , 'OK', 'X', 4, 'XITB'],
 ['OK_D5_BRDA'   , 'OK', 'D', 5, 'BRDA'],
 ['OK_W5_BRDA'   , 'OK', 'W', 5, 'BRDA'],
 ['OK_5_XITB'    , 'OK', 'X', 5, 'XITB'],
 ['OK_D6_BRDA'   , 'OK', 'D', 6, 'BRDA'],
 ['OK_W6_BRDA'   , 'OK', 'W', 6, 'BRDA'],
 ['OK_6_XITB'    , 'OK', 'X', 6, 'XITB'],
 ['OK_D7_BRDA'   , 'OK', 'D', 7, 'BRDA'],
 ['OK_W7_BRDA'   , 'OK', 'W', 7, 'BRDA'],
 ['OK_7_XITB'    , 'OK', 'X', 7, 'XITB'],
 ['OK_D8_BRDA'   , 'OK', 'D', 8, 'BRDA'],
 ['OK_W8_BRDA'   , 'OK', 'W', 8, 'BRDA'],
 ['OK_8_XITB'    , 'OK', 'X', 8, 'XITB'],
 ['OK_D9_BRDA'   , 'OK', 'D', 9, 'BRDA'],
 ['OK_W9_BRDA'   , 'OK', 'W', 9, 'BRDA'],
 ['OK_9_XITB'    , 'OK', 'X', 9, 'XITB']
], columns=('TransitVariable','TimeOfDay','AccessMode','HeirarchyMode','BrdAXitB'))
dfVarToFields

,TransitVariable,TimeOfDay,AccessMode,HeirarchyMode,BrdAXitB
0,PK_D4_BRDA,PK,D,4,BRDA
1,PK_W4_BRDA,PK,W,4,BRDA
2,PK_4_XITB,PK,X,4,XITB
3,PK_D5_BRDA,PK,D,5,BRDA
4,PK_W5_BRDA,PK,W,5,BRDA
5,PK_5_XITB,PK,X,5,XITB
6,PK_D6_BRDA,PK,D,6,BRDA
7,PK_W6_BRDA,PK,W,6,BRDA
8,PK_6_XITB,PK,X,6,XITB
9,PK_D7_BRDA,PK,D,7,BRDA


In [139]:
dfPALong = pd.DataFrame.merge(dfPADetail_melt, dfVarToFields, on='TransitVariable')
dfPALong.drop(columns=('TransitVariable'),inplace=True)
dfPALong_pivot = dfPALong.pivot_table(index=('scenarioID','NAME','Direction','MODE','LINKSEQ','A','B','STOPA','STOPB','TimeOfDay','HeirarchyMode'), columns=('AccessMode','BrdAXitB'))
dfPALong_pivot.columns = dfPALong_pivot.columns.to_flat_index()
dfPALong_pivot = dfPALong_pivot.reset_index()

# rename some columns
dfPALong_pivot.columns = ('scenarioID','NAME','Direction','MODE','LINKSEQ','A','B','STOPA','STOPB','TimeOfDay','HeirarchyMode','D_BRDA','W_BRDA','X_XITB')

dfPALong_pivot_withIBOB = pd.DataFrame.merge(dfPALong_pivot,dfNetWithSegIDs,on=('A','B'))
dfPALong_pivot_withIBOB = dfPALong_pivot_withIBOB.fillna(0)

In [142]:
# CALCULATE RIDERS BY LINK

# column denifinitions
_keyColumns = ['scenarioID','TimeOfDay','MODE','NAME','Direction','HeirarchyMode']
_sortColumns = _keyColumns + ['LINKSEQ']

# sort by for transit direcition
dfPAPivot = dfPALong_pivot_withIBOB.sort_values(by=_sortColumns)

# shift exits to next link in order to facilitate rider calcs
dfPAPivot['X_XITB_Shift'] = dfPAPivot['X_XITB'].shift(1)

# set first link seq to zero because shift would have moved from previous route/time/direction/etc.
dfPAPivot.loc[dfPAPivot['LINKSEQ']==1,'X_XITB_Shift'] = 0

# calculate cummulative sums by key columns
dfPAPivot['D_BRDA_cumsum'      ] = dfPAPivot.groupby(_keyColumns)['D_BRDA'      ].transform(pd.Series.cumsum)
dfPAPivot['W_BRDA_cumsum'      ] = dfPAPivot.groupby(_keyColumns)['W_BRDA'      ].transform(pd.Series.cumsum)
dfPAPivot['X_XITB_Shift_cumsum'] = dfPAPivot.groupby(_keyColumns)['X_XITB_Shift'].transform(pd.Series.cumsum)

# riders is total of cumulative drive and walk boardings minus the shift in exits
dfPAPivot['Riders'] = dfPAPivot['D_BRDA_cumsum'] + dfPAPivot['W_BRDA_cumsum'] - dfPAPivot['X_XITB_Shift_cumsum']

In [143]:
dfPAPivot

,scenarioID,NAME,Direction,MODE,LINKSEQ,A,B,STOPA,STOPB,TimeOfDay,...,D_BRDA,W_BRDA,X_XITB,SEGID,IB_OB,X_XITB_Shift,D_BRDA_cumsum,W_BRDA_cumsum,X_XITB_Shift_cumsum,Riders
1464,1,UT806,P,4,86,22299,22301,0,0,OK,...,0.0,0.00,0.00,WFRC_8222,IB,NaN,0.00,0.00,NaN,NaN
1465,1,UT806,P,4,86,22299,22301,0,0,OK,...,0.0,0.00,0.00,WFRC_8222,IB,0.00,0.00,0.00,0.00,0.00
1466,1,UT806,P,4,86,22299,22301,0,0,OK,...,0.0,0.00,0.00,WFRC_8222,IB,0.00,0.00,0.00,0.00,0.00
1467,1,UT806,P,4,86,22299,22301,0,0,OK,...,0.0,0.00,0.00,WFRC_8222,IB,0.00,0.00,0.00,0.00,0.00
1468,1,UT806,P,4,86,22299,22301,0,0,OK,...,0.0,0.00,0.00,WFRC_8222,IB,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569359,32,WE603,P,9,36,28727,28678,1,1,PK,...,0.0,4.83,1.15,0203_002.3,OB,482.29,138.56,313.32,515.18,-63.30
1570499,32,WE603,P,9,37,28678,28651,1,1,PK,...,0.0,0.21,25.19,0203_001.8,OB,1.15,138.56,313.53,516.33,-64.24
1571639,32,WE603,P,9,38,28651,28631,1,1,PK,...,0.0,0.17,0.30,0203_001.8,OB,25.19,138.56,313.70,541.52,-89.26
1572779,32,WE603,P,9,39,28631,28619,1,0,PK,...,0.0,0.01,0.00,0203_001.6,OB,0.30,138.56,313.71,541.82,-89.55


In [144]:
dfPARiders = dfPAPivot.drop(columns=(['D_BRDA_cumsum','W_BRDA_cumsum','X_XITB_Shift_cumsum']))
dfPARiders['LinkID'] = dfPARiders['A'].astype(str) + '_' + dfPARiders['B'].astype(str)
dfPARiders = dfPARiders[['scenarioID','TimeOfDay','MODE','NAME','HeirarchyMode','LinkID','IB_OB','SEGID','Riders']]
dfPARiders['IB_OB'] = dfPARiders['IB_OB'].fillna('XX')
dfPARiders['SEGID'] = dfPARiders['SEGID'].fillna('')
dfPARiders

,scenarioID,TimeOfDay,MODE,NAME,HeirarchyMode,LinkID,IB_OB,SEGID,Riders
1464,1,OK,4,UT806,4,22299_22301,IB,WFRC_8222,NaN
1465,1,OK,4,UT806,5,22299_22301,IB,WFRC_8222,0.00
1466,1,OK,4,UT806,6,22299_22301,IB,WFRC_8222,0.00
1467,1,OK,4,UT806,7,22299_22301,IB,WFRC_8222,0.00
1468,1,OK,4,UT806,8,22299_22301,IB,WFRC_8222,0.00
...,...,...,...,...,...,...,...,...,...
1569359,32,PK,9,WE603,9,28727_28678,OB,0203_002.3,-63.30
1570499,32,PK,9,WE603,9,28678_28651,OB,0203_001.8,-64.24
1571639,32,PK,9,WE603,9,28651_28631,OB,0203_001.8,-89.26
1572779,32,PK,9,WE603,9,28631_28619,OB,0203_001.6,-89.55


# Boardings/Alightings by Node

In [145]:
# calculate boardings
dfPABoardings  = dfPAPivot.groupby(_keyColumns + ['A','IB_OB','SEGID'],as_index=False).agg(D_BRD=('D_BRDA','sum'),W_BRD=('W_BRDA','sum'))
dfPABoardings = dfPABoardings[(dfPABoardings['D_BRD']>0) | (dfPABoardings['W_BRD']>0)]
#display(dfPABoardings)

# calculate alightings (exits)
dfPAAlightings = dfPAPivot.groupby(_keyColumns + ['B','IB_OB','SEGID'],as_index=False).agg(X_XIT=('X_XITB','sum'))
dfPAAlightings = dfPAAlightings[(dfPAAlightings['X_XIT']>0)]
#display(dfPAAlightings)

# combine
dfPABoardAlight = pd.DataFrame.merge(dfPABoardings, dfPAAlightings, left_on=(_keyColumns + ['A','IB_OB','SEGID']), right_on=(_keyColumns + ['B','IB_OB','SEGID']), how='outer')
dfPABoardAlight = dfPABoardAlight.fillna(0)
dfPABoardAlight['STOPNODE'] = dfPABoardAlight[['A', 'B']].max(axis=1).astype(int)
dfPABoardAlight = dfPABoardAlight[['scenarioID','TimeOfDay','MODE','NAME','IB_OB','SEGID','HeirarchyMode','STOPNODE','D_BRD','W_BRD','X_XIT']]
dfPABoardAlight

,scenarioID,TimeOfDay,MODE,NAME,IB_OB,SEGID,HeirarchyMode,STOPNODE,D_BRD,W_BRD,X_XIT
0,1,OK,4,UT834,OB,2218_000.5,4,23869,0.0,1.97,0.00
1,1,OK,4,UT834,OB,2218_000.5,8,23869,0.0,0.23,0.00
2,1,OK,4,UT834,OB,2218_000.5,9,23869,0.0,0.80,0.00
3,1,OK,4,UT862,OB,2080_005.7,4,22541,0.0,2.64,0.00
4,1,OK,4,UT862,IB,2080_005.7,4,22540,0.0,5.43,0.00
...,...,...,...,...,...,...,...,...,...,...,...
599055,32,PK,9,WE603,OB,3354_002.6,9,28820,0.0,0.00,0.37
599056,32,PK,9,WE603,IB,3400_000.3,9,28823,0.0,0.00,3.96
599057,32,PK,9,WE603,OB,3400_000.6,9,28899,0.0,0.00,2.38
599058,32,PK,9,WE603,IB,3404_001.5,9,28935,0.0,0.00,0.91


In [146]:
dfPABoardAlight.to_csv('pa_board_alight_test.csv')

In [156]:
os.path.join(_outputdir_paLink,_file)

'e:\\GitHub\\TDM-Scenario-Comparison-Toolbox\\2.1-TotalTDMTool\\_TTTDataTransitPALink\\id_12066090786226718a9b4755.54851854.json'

In [160]:
dfJoin

,scenarioID,value
0,1,id_12066090786226718a9b4755.54851854
1,2,id_602780967622672681e6327.84737895
2,3,id_4927651456226727eb56079.08455813
3,4,id_16231621306226729673c147.07236076
4,5,id_3989814362267228a9f4b5.00876734
5,18,id_37001244863cadd40e1b8f7.52642706
6,19,id_72534583163cb54226ea202.98630416
7,20,id_73730688563cb5d2222cdc4.25550859
8,21,id_71963308763cb5d35348012.40299726
9,22,id_23888985563cb5d49295ba5.99558740


In [159]:
export = True

_outputdir_paLink = os.path.join(os.getcwd(),'_TTTDataTransitPALink')
_outputdir_paNode = os.path.join(os.getcwd(),'_TTTDataTransitPANode')

# loop through scenarios and calc Link A and B -- USE Transit-Map TransitRidership.ipynb methodology
for index, row in dfScenarios.iterrows():

    try:
        print( dfJoin[dfJoin['scenarioID']==row['scenarioID']]['value'].tolist())
        _id = dfJoin[dfJoin['scenarioID']==row['scenarioID']]['value'].tolist()[0]
        #print(_id)
        _file = _id + '.json'
        #print(_file)
        _outputfile_paLink = os.path.join(_outputdir_paLink,_file)
        _outputfile_paNode = os.path.join(_outputdir_paNode,_file)

        dfLinkOut = dfPARiders[dfPARiders['scenarioID']==row['scenarioID']]
        dfLinkOut = dfLinkOut.drop(columns=('scenarioID'))
        dfLinkOut = dfLinkOut.set_index('SEGID')
        if export: dfModeOut.to_json(_outputfile_paLink,orient="table") #,orient='records'

        dfNodeOut = dfPABoardAlight[dfPABoardAlight['scenarioID']==row['scenarioID']]
        dfNodeOut = dfNodeOut.drop(columns=('scenarioID'))
        dfNodeOut = dfNodeOut.set_index('SEGID')
        if export: dfNodeOut.to_json(_outputfile_paNode,orient="table") #,orient='records'
        #display(dfTemp)
#    except:
#        print('scenarioID ' + str(row['scenarioID']) + ' error')
#        continue

IndexError: list index out of range

In [150]:
ids = dfToJson["ID"]
dfToJson[ids.isin(ids[ids.duplicated()])].sort_values("ID")

,ID,Riders


In [149]:
dfToJson = dfPARiders[dfPARiders['scenarioID']==1].copy()
dfToJson['ID'] = dfToJson['LinkID'] + "_" + dfToJson['TimeOfDay'] + "_" + dfToJson['MODE'].astype(str) + "_" + dfToJson['NAME'] + "_" + dfToJson['HeirarchyMode'].astype(str) + "_" + dfToJson['IB_OB']
dfToJson = dfToJson[['ID','Riders']]
dfToJson = dfToJson.drop_duplicates()
print (dfToJson.set_index('ID').rename(columns={'Riders':'index1'}).to_json())
#{"index1":{"1":"a","2":"b","3":"c","4":"d"}}




{"index1":{"22299_22301_OK_4_UT806_4_IB":null,"22299_22301_OK_4_UT806_5_IB":0.0,"22299_22301_OK_4_UT806_6_IB":0.0,"22299_22301_OK_4_UT806_7_IB":0.0,"22299_22301_OK_4_UT806_8_IB":0.0,"22299_22301_OK_4_UT806_9_IB":0.0,"23869_23870_OK_4_UT834_4_OB":1.97,"23869_23870_OK_4_UT834_5_OB":0.0,"23869_23870_OK_4_UT834_6_OB":0.0,"23869_23870_OK_4_UT834_7_OB":0.0,"23869_23870_OK_4_UT834_8_OB":0.23,"23869_23870_OK_4_UT834_9_OB":0.8,"23870_23869_OK_4_UT834_4_IB":0.0,"23870_23869_OK_4_UT834_5_IB":-13.39,"23870_23869_OK_4_UT834_6_IB":0.0,"23870_23869_OK_4_UT834_7_IB":0.0,"23870_23869_OK_4_UT834_8_IB":0.0,"23870_23869_OK_4_UT834_9_IB":-0.79,"22541_22540_OK_4_UT862_4_OB":2.18,"22541_22540_OK_4_UT862_5_OB":-9.88,"22541_22540_OK_4_UT862_6_OB":0.0,"22541_22540_OK_4_UT862_7_OB":0.0,"22541_22540_OK_4_UT862_8_OB":0.0,"22541_22540_OK_4_UT862_9_OB":0.0,"22540_22541_OK_4_UT862_4_IB":5.43,"22540_22541_OK_4_UT862_5_IB":-0.56,"22540_22541_OK_4_UT862_6_IB":0.0,"22540_22541_OK_4_UT862_7_IB":0.0,"22540_22541_OK_4_UT862

In [101]:
dfPARiders

,scenarioID,TimeOfDay,MODE,NAME,Direction,HeirarchyMode,LinkID,IB_OB,SEGID,Riders
4584,1,OK,4,BE630,N,4,20142_6885,,,13.28
4596,1,OK,4,BE630,N,4,6885_6923,,,13.28
4608,1,OK,4,BE630,N,4,6923_6888,,,13.29
4620,1,OK,4,BE630,N,4,6888_9357,,,13.44
4632,1,OK,4,BE630,N,4,9357_6889,,,13.44
...,...,...,...,...,...,...,...,...,...,...
4672223,32,PK,9,WE603,P,9,28727_28678,OB,0203_002.3,35.79
4672235,32,PK,9,WE603,P,9,28678_28651,OB,0203_001.8,34.85
4672247,32,PK,9,WE603,P,9,28651_28631,OB,0203_001.8,9.83
4672259,32,PK,9,WE603,P,9,28631_28619,OB,0203_001.6,9.54
